In [1]:
from Angrybird import AngryBird
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf
from tqdm.notebook import tqdm

In [2]:
db1=np.load("db1.npy")
db2=np.load("db2.npy")
ub1=np.load("ub1.npy")
ub2=np.load("ub2.npy")
print(ub1 - db1)
print(ub2 - db2)

[  3.3702401    9.64839558  18.13527645  26.98632257  38.0141466
  50.74219191  65.30956424  81.90413466 103.90807588]
[  3.86655689   9.40276866  17.21078777  28.23815127  40.31978872
  53.67959175  67.17050199  83.13494255 104.10694185]


In [3]:
# Polynome de deg 3
def init_mat():
    fmat = []
    fmat.append(np.zeros((1, 1, 2)))
    for i in range(1, 10):
        fmat.append(np.zeros((i, i, 8)))
    return fmat

def force_from_coeff(x1, x2, coeffs):
    x1 /= 100
    x2 /= 50
    a0, a1, a2, a3, b0, b1, b2, b3 = coeffs
    force1 = a0 + a1*x1 + a2*x1**2 + a3*x1**3
    force2 = b0 + b1*x2 + b2*x2**2 + b3*x2**3
    return np.array([force1, force2])

def force_from_mat(mat, state):
    t, x1, x2 = state
    t = int(t)
    if t == 0:
        return mat[0][0][0]
    else:
        i1 = min(max(int((x1 - db1[t-1]) / (ub1[t-1]-db1[t-1]) * t), 0), t-1)
        i2 = min(max(int((x2 - db2[t-1]) / (ub2[t-1]-db2[t-1]) * t), 0), t-1)
        #return mat[t][i1][i2]
        #print(i1, i2)
        return force_from_coeff(x1, x2, mat[t][i1][i2])

# Force de rappel
def main(state):
    t = state[0]
    return 1.59 * (-1.0125*state[1:]+np.array([10*t, 20*t-2*t**2]))

def rand_place():
    i = np.random.randint(0, 2282)
    #print(i)
    if i < 2:
        return 0, 0, 0, i
    else:
        j = 2
        for a in range(1, 10):
            for b in range(a):
                for c in range(a):
                    for d in range(8):
                        if i == j:
                            return a, b, c, d
                        else:
                            j += 1
    print(i)

def rand_places(i):
    places = []
    for _ in range(i):
        places.append(rand_place())
    return places

def tups_update(mat, places, lr, j):
    try:
        for tup in places:
            a, b, c, d = tup
            mat[a][b][c][d] += lr * j
    except:
        print(places)

In [4]:
umat = init_mat()
#umat = np.load("umat.npy")
vmat = init_mat()

In [5]:
#np.save("vents", vents)
vents = np.load("vents.npy")

In [ ]:
env = AngryBird()
#env2 = AngryBird()
restart = True
lr = 0.01
m = 10000
if restart:
    step = 1
    all_rewards = []
rewards = np.zeros(m)
average_reward = 300.
while average_reward > 110.:
    if (step+1)%10==0:
        print("Step {}, last cost = {}, mean over 100 rounds = {}".format(step+1, average_reward, np.mean(all_rewards[-100:])))
        np.save("rewards", all_rewards)
#    lr = 1 / (np.log10(step) / 2 + 1)
    places = rand_places(10)
    tups_update(vmat, places, lr, 1)
    cost1 = 0.
    #cost2 = 0.
    j_ = 0.
    for z in range(m):
        state = env.reset()
        #state2 = env2.reset()
        env.vent=vents[z]
        #env2.vent=vents[z]
        for i in range(10):
            u_force = force_from_mat(umat, state) + main(state)
            v_force = force_from_mat(vmat, state)
            state, _, _, _ = env.step(u_force, v_force)
            #state2, _, _, _ = env2.step(v_force)
        env.step(np.zeros(2))
        #env2.step(np.zeros(2))
        cost1 += np.sum(env.cost)
        #cost2 += np.sum(env2.cost)
        rewards[z] = np.sum(env.cost)
        j_ += env.j_prime_MC()
    tups_update(umat, places, -j_/m * lr, 1)
    #print(j_ / m)
    tups_update(vmat, places, lr, -1)
    step+=1
    average_reward = np.mean(rewards)
    all_rewards.append(average_reward)

Step 10, last cost = 120.26125899921549, mean over 100 rounds = 120.26133648131989
Step 20, last cost = 120.26105672746989, mean over 100 rounds = 120.26122179504634
Step 30, last cost = 120.26106461844621, mean over 100 rounds = 120.26116439765062
Step 40, last cost = 120.2608485793259, mean over 100 rounds = 120.26111060355106
Step 50, last cost = 120.26079238101337, mean over 100 rounds = 120.2610502790706
Step 60, last cost = 120.26090010549885, mean over 100 rounds = 120.26102025729243
Step 70, last cost = 120.26097971150882, mean over 100 rounds = 120.26101165414116
Step 80, last cost = 120.26093676125063, mean over 100 rounds = 120.2610098807764
Step 90, last cost = 120.26097334847233, mean over 100 rounds = 120.26100329670341
Step 100, last cost = 120.26092340718408, mean over 100 rounds = 120.26100017618215
Step 110, last cost = 120.26069965393185, mean over 100 rounds = 120.26095247902471
Step 120, last cost = 120.2607228126361, mean over 100 rounds = 120.26090998931485
Step 

#### Save
db1, ub1, db2, ub2, umat

In [ ]:
umat = np.array(umat, dtype=object)
np.save("umat", umat)